In [2]:
import pandas as pd
import numpy as np
import spacy
import unicodedata
from multiprocessing import  Pool
import re

pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

In [3]:
def parallelize_dataframe(df, func, n_cores=12):
    """
    Function that parallelizes any function applied to a dataframe
    Input:
    df      - Dataframe
    func    - Function to be aplied to portions of Dataframe
    n_cores - Number of CPU cores to be used on the application of func
    
    """
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [5]:
# Load stopwords from file
stopwords = open("stopwords.txt", "r")
stopwords = stopwords.read()
stopwords = stopwords.splitlines()

#%%
def clean_stopwords(text):
    text = text.lower()
    text = text.split()
    #text = text.dropna() # remove nan (not a number)

    #exp = r"\/|\[|\(|\)|\.|-|\]|\+"
    #text = re.sub(exp, '', text)
    #text = re.sub("Þþ", '', text)
    

    lista = []
    for word in text:
        word = re.sub(r'\b\/\b', r' ', word)
        #word = re.sub(r'\/', r' ', word)
        word = re.sub(r'\b[,;]', '', word)
        word = re.sub(r'nan', '', word)
        #word = re.sub(r"[/]", ' ', word)
        #word = re.sub(r"\W", r'', word) # remove non-letter characters
        word = re.sub(r"\d", r'', word) # remove digits
        if word in stopwords:
            pass
        else:
            lista.append(word)

    return lista

def normaliza(old):
    new = ''.join(ch for ch in unicodedata.normalize('NFKD', 
                    old) if not unicodedata.combining(ch))
    
    return new

def do_norm(l_strings):        
    lista = []
    for t in l_strings:
        lista.append(normaliza(t))
    
    return lista

def preprocessing(df):
    return df.apply(lambda x: " ".join(clean_stopwords(normaliza(' '.join(x.astype(str))))), axis=1)

In [7]:
df = pd.read_csv("estaticos_market.csv",encoding='utf_8')
df.head(5)

,Unnamed: 0,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_spa,fl_antt,fl_veiculo,vl_total_tancagem,vl_total_veiculos_antt,vl_total_veiculos_leves,vl_total_veiculos_pesados,fl_optante_simples,qt_art,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,vl_total_tancagem_grupo,vl_total_veiculos_antt_grupo,vl_potenc_cons_oleo_gas,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,nu_meses_rescencia,de_nivel_atividade,de_indicador_telefone,fl_simples_irregular,vl_frota,empsetorcensitariofaixarendapopulacao,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,idade_maxima_socios,idade_minima_socios,qt_socios_st_regular,qt_socios_st_suspensa,qt_socios_masculino,qt_socios_feminino,qt_socios_pep,qt_alteracao_socio_total,qt_alteracao_socio_90d,qt_alteracao_socio_180d,qt_alteracao_socio_365d,qt_socios_pj_ativos,qt_socios_pj_nulos,qt_socios_pj_baixados,qt_socios_pj_suspensos,qt_socios_pj_inaptos,vl_idade_media_socios_pj,vl_idade_maxima_socios_pj,vl_idade_minima_socios_pj,qt_coligados,qt_socios_coligados,qt_coligados_matriz,qt_coligados_ativo,qt_coligados_baixada,qt_coligados_inapta,qt_coligados_suspensa,qt_coligados_nula,idade_media_coligadas,idade_maxima_coligadas,idade_minima_coligadas,coligada_mais_nova_ativa,coligada_mais_antiga_ativa,idade_media_coligadas_ativas,coligada_mais_nova_baixada,coligada_mais_antiga_baixada,idade_media_coligadas_baixadas,qt_coligados_sa,qt_coligados_me,qt_coligados_mei,qt_coligados_ltda,qt_coligados_epp,qt_coligados_norte,qt_coligados_sul,qt_coligados_nordeste,qt_coligados_centro,qt_coligados_sudeste,qt_coligados_exterior,qt_ufs_coligados,qt_regioes_coligados,qt_ramos_coligados,qt_coligados_industria,qt_coligados_agropecuaria,qt_coligados_comercio,qt_coligados_serviço,qt_coligados_ccivil,qt_funcionarios_coligados,qt_funcionarios_coligados_gp,media_funcionarios_coligados_gp,max_funcionarios_coligados_gp,min_funcionarios_coligados_gp,vl_folha_coligados,media_vl_folha_coligados,max_vl_folha_coligados,min_vl_folha_coligados,vl_folha_coligados_gp,media_vl_folha_coligados_gp,max_vl_folha_coligados_gp,min_vl_folha_coligados_gp,faturamento_est_coligados,media_faturamento_est_coligados,max_faturamento_est_coligados,min_faturamento_est_coligados,faturamento_est_coligados_gp,media_faturamento_est_coligados_gp,max_faturamento_est_coligados_gp,min_faturamento_est_coligados_gp,total_filiais_coligados,media_filiais_coligados,max_filiais_coligados,min_filiais_coligados,qt_coligados_atividade_alto,qt_coligados_atividade_medio,qt_coligados_atividade_baixo,qt_coligados_atividade_mt_baixo,qt_coligados_atividade_inativo,qt_coligadas,sum_faturamento_estimado_coligadas,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_ex_funcionarios,qt_funcionarios_grupo,percent_func_genero_masc,percent_func_genero_fem,idade_ate_18,idade_de_19_a_23,idade_de_24_a_28,idade_de_29_a_33,idade_de_34_a_38,idade_de_39_a_43,idade_de_44_a_48,idade_de_49_a_53,idade_de_54_a_58,idade_acima_de_58,grau_instrucao_macro_analfabeto,grau_instrucao_macro_escolaridade_fundamental,grau_instrucao_macro_escolaridade_media,grau_instrucao_macro_escolaridade_superior,grau_instrucao_macro_desconhecido,total,meses_ultima_contratacaco,qt_admitidos_12meses,qt_desligados_12meses,qt_desligados,qt_admitidos,media_meses_servicos_all,max_meses_servicos_all,min_meses_servicos_all,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
0,0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,True,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,INDUSTRIA DA CONSTRUCAO,CONSTRUÇÃO CIVIL,14.457534,10 a 15,False,Fal

In [9]:
columns_to_keep = ["id","de_natureza_juridica","natureza_juridica_macro",
                   "de_ramo","setor","nm_divisao","nm_segmento","nm_meso_regiao"]
df_cols = df[columns_to_keep].copy()

In [11]:
df_cols.head(5)

,id,de_natureza_juridica,natureza_juridica_macro,de_ramo,setor,nm_divisao,nm_segmento,nm_meso_regiao
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,SOCIEDADE EMPRESARIA LIMITADA,ENTIDADES EMPRESARIAIS,INDUSTRIA DA CONSTRUCAO,CONSTRUÇÃO CIVIL,CONSTRUCAO DE EDIFICIOS,CONSTRUCAO,LESTE POTIGUAR
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,EMPRESARIO INDIVIDUAL,OUTROS,SERVICOS DE ALOJAMENTO/ALIMENTACAO,SERVIÇO,ALIMENTACAO,ALOJAMENTO E ALIMENTACAO,CENTRO NORTE PIAUIENSE
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,EMPRESARIO INDIVIDUAL,OUTROS,"TRANSPORTE, ARMAZENAGEM E CORREIO",SERVIÇO,TRANSPORTE TERRESTRE,TRANSPORTE ARMAZENAGEM E CORREIO,CENTRO AMAZONENSE
3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,EMPRESARIO INDIVIDUAL,OUTROS,SERVICOS DIVERSOS,SERVIÇO,REPARACAO E MANUTENCAO DE EQUIPAMENTOS DE INFO...,OUTRAS ATIVIDADES DE SERVICOS,CENTRO AMAZONENSE
4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,EMPRESARIO INDIVIDUAL,OUTROS,"SERVICOS PROFISSIONAIS, TECNICOS E CIENTIFICOS",SERVIÇO,SERVICOS DE ARQUITETURA E ENGENHARIA TESTES E ...,ATIVIDADES PROFISSIONAIS CIENTIFICAS E TECNICAS,LESTE POTIGUAR


In [12]:
identific = df_cols[["id"]].copy()
df_cols.drop(["id"],axis=1,inplace=True)
df2 = parallelize_dataframe(df_cols, preprocessing)
df_final_market = pd.concat([identific,df2], axis=1)
df_final_market.to_csv("MERCADO_FINAL.csv",index=False)

In [13]:
df_final_market[0][16]

'associacao privada entidades fins lucrativos servicos diversos servico atividades organizacoes associativas outras atividades servicos centro norte piauiense'